In [2]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def plot_features_cat_regression(df, target_col="", columns=[], pvalue=0.05, with_individual_plot=False):
    """
    Función para graficar histogramas agrupados de variables categóricas significativas.
    
    Params:
        df: dataframe de pandas
        target_col: columna objetivo del dataframe (variable numérica)
        columns: lista de columnas categóricas a evaluar (si está vacía, se usan todas las columnas categóricas)
        pvalue: p-valor para el test de significancia
        with_individual_plot: si es True, genera un gráfico individual por cada categoría
    
    Returns:
        Lista de columnas que cumplen con los criterios de significancia
    """
    
    # Verificamos si el dataframe es válido
    if not isinstance(df, pd.DataFrame):
        print("El argumento 'df' no es un dataframe válido.")
        return None
    
    # Verificamos si 'target_col' es una columna válida en el dataframe
    if target_col and target_col not in df.columns:
        print(f"La columna '{target_col}' no está en el dataframe.")
        return None
    
    # Verificamos si la columna 'target_col' es numérica
    if target_col and not pd.api.types.is_numeric_dtype(df[target_col]):
        print(f"La columna '{target_col}' no es una columna numérica.")
        return None
    
    # Identificar las columnas categóricas del dataframe
    cat_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    if not cat_columns:
        print("No se encontraron características categóricas en el dataframe.")
        return None

    # Si 'columns' está vacío, usamos todas las columnas categóricas
    if not columns:
        columns = cat_columns
    
    # Lista para almacenar las columnas significativas
    significant_cat_features = []

    # Verificamos la significancia de las columnas categóricas con respecto al 'target_col'
    for cat_col in columns:
        if cat_col not in cat_columns:
            print(f"La columna '{cat_col}' no es categórica o no existe en el dataframe.")
            continue
        
        if df[cat_col].nunique() > 1:
            if df[cat_col].nunique() == 2:
                group1 = df[target_col][df[cat_col] == df[cat_col].unique()[0]]
                group2 = df[target_col][df[cat_col] == df[cat_col].unique()[1]]
                t_stat, p_val = stats.ttest_ind(group1, group2)
            else:
                f_val, p_val = stats.f_oneway(*[df[target_col][df[cat_col] == level] for level in df[cat_col].unique()])
            
            # Comprobamos si el p-valor es menor que el p-valor especificado
            if p_val < pvalue:
                significant_cat_features.append(cat_col)
                
    # Si no hay columnas significativas
    if not significant_cat_features:
        print("No se encontraron características categóricas significativas.")
        return None

    # Graficar histogramas agrupados para las columnas significativas
    for cat_col in significant_cat_features:
        plt.figure(figsize=(10, 6))
        sns.histplot(data=df, x=target_col, hue=cat_col, kde=True, multiple="stack", bins=30)
        plt.title(f"Distribución de {target_col} por {cat_col}")
        plt.xlabel(target_col)
        plt.ylabel("Frecuencia")
        plt.show()

        # Si 'with_individual_plot' es True, graficar histogramas individuales por cada categoría
        if with_individual_plot:
            for level in df[cat_col].unique():
                plt.figure(figsize=(10, 6))
                sns.histplot(df[df[cat_col] == level], x=target_col, kde=True, bins=30)
                plt.title(f"Distribución de {target_col} para {cat_col} = {level}")
                plt.xlabel(target_col)
                plt.ylabel("Frecuencia")
                plt.show()

    return significant_cat_features

In [4]:
df = pd.read_csv('../data/student_performance_data.csv')

In [5]:
plot_features_cat_regression(df, target_col='GPA', pvalue=0.05, with_individual_plot=False)

No se encontraron características categóricas en el dataframe.
